# LFADS
This notebook is part of the [SachsLab Workshop for Intracranial Neurophysiology and Deep Learning](https://github.com/SachsLab/IntracranialNeurophysDL).

Follow the link below to run in Google Colab or continue if running locally.

Run the first few cells to normalize Local / Colab environments, then proceed below for the lesson.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/SachsLab/IntracranialNeurophysDL/blob/master/notebooks/06_01_LFADS.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/SachsLab/IntracranialNeurophysDL/blob/master/notebooks/06_01_LFADS.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [1]:
import os
import sys
from pathlib import Path
try:
    # See if we are running on google.colab
    import google.colab
    from google.colab import files
    if sys.version_info > (3, 0):
        from importlib import reload
        !pip install --upgrade -q https://storage.googleapis.com/jax-wheels/cuda$(echo $CUDA_VERSION | sed -e 's/\.//' -e 's/\..*//')/jaxlib-latest-cp36-none-linux_x86_64.whl
    else:
        !pip install --upgrade -q https://storage.googleapis.com/jax-wheels/cuda$(echo $CUDA_VERSION | sed -e 's/\.//' -e 's/\..*//')/jaxlib-latest-cp27-none-linux_x86_64.whl
    !pip install --upgrade -q git+https://github.com/google/jax.git

    os.chdir('..')
    if not (Path.home() / '.kaggle').is_dir():
        # Configure kaggle
        files.upload()  # Find the kaggle.json file in your ~/.kaggle directory.
        !pip install -q kaggle
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
    if Path.cwd().stem != 'IntracranialNeurophysDL':
        if not (Path.cwd() / 'IntracranialNeurophysDL').is_dir():
            # Download the workshop repo and change to its directory
            !git clone --recursive https://github.com/SachsLab/IntracranialNeurophysDL.git
        os.chdir('IntracranialNeurophysDL')
    IN_COLAB = True
except ModuleNotFoundError:
    IN_COLAB = False
    if Path.cwd().stem == 'notebooks':
        os.chdir(Path.cwd().parent)
    # Make sure the kaggle executable is on the PATH
    os.environ['PATH'] = os.environ['PATH'] + ';' + str(Path(sys.executable).parent / 'Scripts')

In [2]:
# Download and unzip data (2.1 GB)
datadir = Path.cwd() / 'data' / 'joeyo'
if not (datadir / 'converted').is_dir():
    !kaggle datasets download --unzip --path {str(datadir / 'converted')} cboulay/joeyo_nhp_reach_mea
    print("Finished downloading and extracting data.")
else:
    print("Data directory found. Skipping download.")

Data directory found. Skipping download.


## Imports

In [4]:
import jax.numpy as np
from jax import jit, random, vmap, grad, lax
from jax.experimental import optimizers
#config.update("jax_debug_nans", True) # Useful for finding numerical errors
import matplotlib.pyplot as plt
import numpy as onp  # original CPU-backed NumPy
import time

## Get Data
Unlike notebook 05_02, where we used a configurable but small number of wide bins per sequence,
here we use a large number of narrow bins per sequence, where a sequence corresponds to an entire trial
from target onset to target hit (and a bit beyond).
### Data Hyperparameters

In [5]:
SESS_IDX = 7          # Index of recording session we will use. 0:8
MAX_TRIAL_DUR = 1.7   # This gets rid of about 7% of the slowest trials (long tail distribution)
BIN_DURATION = 0.020  # Width of window used to bin spikes, in seconds
P_TRAIN = 0.8         # Proportion of data used for training.

### Load Spiking Data

In [10]:
from data.utils.fileio import load_joeyo_reaching

def load_dat_with_vel_accel(datadir, sess_idx, x_chunk='mu_spiketimes', trial_dur=1.7):
    BEHAV_CHANS = ['CursorX', 'CursorY']
    sess_names = ['indy_201' + _ for _ in ['60921_01', '60927_04', '60927_06', '60930_02', '60930_05', '61005_06',
                                           '61006_02', '70124_01', '70127_03']]
    X, Y, X_ax_info, Y_ax_info = load_joeyo_reaching(datadir, sess_names[sess_idx], x_chunk=x_chunk)

    # Determine target onset times that we will keep.
    targ_ch_ix = onp.where(onp.in1d(Y_ax_info['channel_names'], ['TargetX', 'TargetY']))[0]
    b_targ_onset = onp.hstack((True, onp.any(onp.diff(Y[targ_ch_ix]) != 0, axis=0)))
    targ_onset_times = Y_ax_info['timestamps'][b_targ_onset]
    targ_onset_times = targ_onset_times[onp.hstack((onp.diff(targ_onset_times) <= trial_dur, False))]

    # Slice Y to only keep required behaviour data (cursor position)
    b_keep_y_chans = onp.in1d(Y_ax_info['channel_names'], BEHAV_CHANS)
    Y = Y[b_keep_y_chans, :]
    Y_ax_info['channel_names'] = [_ for _ in Y_ax_info['channel_names'] if _ in BEHAV_CHANS]

    # Calculate discrete derivative and double-derivative to get velocity and acceleration.
    vel = onp.diff(Y, axis=1)
    vel = onp.concatenate((vel[:, 0][:, None], vel),
                         axis=1)  # Assume velocity was constant across the first two samples.
    accel = onp.concatenate(([[0], [0]], np.diff(vel, axis=1)), axis=1)  # Assume accel was 0 in the first sample.
    Y = onp.concatenate((Y, vel, accel), axis=0)
    Y_ax_info['channel_names'] += ['VelX', 'VelY', 'AccX', 'AccY']

    return X, Y, X_ax_info, Y_ax_info, targ_onset_times

X, Y, X_ax_info, Y_ax_info, targ_times = load_dat_with_vel_accel(datadir, SESS_IDX, trial_dur=MAX_TRIAL_DUR)


ValueError: Must use the three-argument form of where().

### Segment Into Trials 

In [ ]:
def bin_and_segment_spike_times(X, X_ax_info, targ_onset_times,
                                nearest_bin_dur=0.005, nearest_bin_step_dur=None, trial_dur=1.7):

    # We'll use an integer number of samples per bin: the next highest required to get the requested nearest_bin_dur
    samps_per_bin = int(onp.ceil(nearest_bin_dur * X_ax_info['fs']))
    true_bin_dur = samps_per_bin / X_ax_info['fs']
    print("Actual bin duration: {}".format(true_bin_dur))

    # Similar for number of samples per bin-step.
    if nearest_bin_step_dur is not None:
        samps_per_step = int(onp.ceil(nearest_bin_step_dur * X_ax_info['fs']))
        print("Actual bin duration: {}".format(samps_per_bin / X_ax_info['fs']))
    else:
        # No overlap
        samps_per_step = samps_per_bin

    # Get the indices of each bin-start
    bin_starts_idx = onp.arange(0, X.shape[-1], samps_per_step)
    b_full_bins = bin_starts_idx <= (X.shape[-1] - samps_per_bin)
    bin_starts_idx = bin_starts_idx[b_full_bins]

    # The next chunk of code counts the number of spikes in each bin.
    # -Create array of indices to reslice the raster data
    bin_ix = onp.arange(samps_per_bin)[:, None] + bin_starts_idx[None, :]
    # -Create buffer to hold the dense raster data
    _temp = onp.zeros(X[0].shape, dtype=bool)
    # -Preallocate _X to hold spike counts per bin
    _X = onp.zeros((len(bin_starts_idx), X.shape[0]), dtype=np.int32)
    for chan_ix in range(X.shape[0]):
        _X[:, chan_ix] = np.sum(X[chan_ix].toarray(out=_temp)[0][bin_ix], axis=0)

    # Now that our data are binned, let's slice it up into trials.
    bins_per_trial = int(trial_dur / true_bin_dur)
    trial_X = onp.zeros((len(targ_onset_times), bins_per_trial, _X.shape[-1]))

    bin_stops_t = X_ax_info['timestamps'][bin_starts_idx + samps_per_bin - 1]
    trial_starts_idx = onp.searchsorted(bin_stops_t, targ_onset_times)
    for trial_ix, t_start_idx in enumerate(trial_starts_idx):
        trial_X[trial_ix] = _X[t_start_idx:t_start_idx+bins_per_trial, :]

    in_trial_tvec = onp.arange(bins_per_trial) * true_bin_dur

    return trial_X, in_trial_tvec, true_bin_dur

trial_X, trial_tvec, true_bin_dur = bin_and_segment_spike_times(X, X_ax_info, targ_times,
                                                                nearest_bin_dur=BIN_DURATION, trial_dur=MAX_TRIAL_DUR)

### Split Train and Validation

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(trial_X, y, train_size=P_TRAIN)

### Visualize Data

In [ ]:
n_trials, n_timesteps, n_neurons = trial_X.shape
print("trial_X has {} trials, {} bin-steps per trial, and {} neurons.".format(n_trials, n_timesteps, n_neurons))
trial_idx = onp.random.randint(0, n_trials-1, 1)[0]
neuron_idx = onp.random.randint(0, n_neurons-1, 4)
plt.subplot(2, 1, 1)
plt.plot(trial_tvec, trial_X[trial_idx, :, neuron_idx].T)
plt.title("{} random neurons from trial {}".format(len(neuron_idx), trial_idx))
plt.xlabel('Time after target onset (s)')
plt.ylabel('Spikes / Bin')

from sklearn.decomposition import PCA
pca = PCA(5)
pca.fit(onp.reshape(trial_X, (-1, n_neurons)))
plt.subplot(2, 1, 2)
plt.plot(100 - 100*pca.explained_variance_ratio_)
plt.xlabel('Component ID')
plt.ylabel('% VAF')
plt.ylim([0, 100])

# pca_filt_dat = pca.inverse_transform(pca.transform(trial_X[trial_idx, :, :]))
# plt.subplot(3, 1, 3)
# plt.plot(trial_tvec, pca_filt_dat[:, neuron_idx], '-')
plt.show()

## LFADS - Latent Factor Analysis via Dynamical Systems
This notebook is based on a notebook found in in the [google-research/computation-thru-dynamics github repo](https://github.com/google-research/computation-thru-dynamics).

In the below diagram, the "Generator" is the cortical neuronal population, which is assumed to be a nonlinear,
dynamical system that is modeled with a RNN. For now we will ignore the bottom "Controller" and "Inferred inputs" parts.

![](https://raw.githubusercontent.com/google-research/computation-thru-dynamics/master/images/lfads_architecture_w_inferred_inputs_3.png)

The data is put through nonlinear, recurrent **encoders**, and this produces an **initial state distribution**,
which is a per-trial mean and variance to produce random vectors to encode that trial.
This is exactly the same as the 'bottleneck' or 'latent variables' we saw in the variational auto-encoder tutorial.

The initial state of the generator is a randomly drawn vector from this distribution.
The **generator** marches through time and at each time point produces "factors" and "rates".
The "factors" are the low-dimensional neural state. The "rates" are the projection of the neural state into neuronal activations.
The rates are then used to parameterize a Poisson process to generate spikes.
The loss function compares the generated spike trains to the original input spike trains.


## LFADS Hyperparameters

In [ ]:
BATCH_SIZE = 4        # Number of trials in each training step during optimization

N_RNN_UNITS = 60      # Size of RNN output (state)
EPOCHS = 10           # Number of loops through the entire data set.

# LFADS architecture - The size of the numbers is rather arbitrary, 
# but relatively small because we know the integrator RNN isn't too high 
# dimensional in its activity.
ENC_DIM = 128         # encoder dim
CON_DIM = 128         # controller dim
II_DIM = 1            # inferred input dim, we know there is 1 dim in integrator RNN
GEN_DIM = 128         # generator dim, should be large enough to generate integrator RNN dynamics
FACTORS_DIM = 32      # factors dim, should be large enough to capture most variance of dynamics

# Numerical stability
VAR_MIN = 0.001       # Minimal variance any gaussian can become.

# Initial state prior parameters
# the mean is set to zero in the code
ic_prior_var = 0.1 # this is sigma^2 of uninformative prior

# Optimization Hyperparameters
# 
num_batches = int(n_trials * EPOCHS / BATCH_SIZE)  # how many batches do we train
print_every = 20            # give information every so often

# Learning rate HPs
step_size = 0.05            # initial learning rate
decay_factor = 0.99985      # learning rate decay param
decay_steps = 1             # learning rate decay param

# Regularization HPs
keep_rate = 0.98            # dropout keep rate during training
P_DROPOUT = 0.05      # Proportion of units to set to 0 on each step.
L2_REG = 2.0e-5       # Parameter regularization strength.

# Numerical stability HPs
max_grad_norm = 10.0        # gradient clipping above this value

# For randomization
MAX_SEED_INT = 10000000


### KL Warmup
Copied straight from [LFADS tutorial optimize.py](https://github.com/google-research/computation-thru-dynamics/blob/master/lfads_tutorial/optimize.py).

> It turns out that the KL term can be a lot easier to optimize initially than learning how to
reconstruct your data. This results in a pathological stoppage of training where the KL goes to
nearly zero and training is broken there on out (as you cannot represent any a given trial from
uninformative priors). One way out of this is to warmup the KL penality, starting it off with a
weight term of 0 and then slowly building to 1, giving the reconstruction a chance to train a bit
without the KL penalty messing things up.

In [ ]:
# The fact that the start and end values are required to be floats is something I need to fix.
kl_warmup_start = 500.0 # batch number to start kl warmup, explicitly float
kl_warmup_end = 1000.0  # batch number to be finished with kl warmup, explicitly float
kl_min = 0.01 # The minimum KL value, non-zero to make sure KL doesn't grow crazy before kicking in.
kl_max = 1.0

def get_kl_warmup_fun():
    def kl_warmup(batch_idx):
        progress_frac = ((batch_idx - kl_warmup_start) /
                         (kl_warmup_end - kl_warmup_start))
        kl_warmup = np.where(batch_idx < kl_warmup_start, kl_min,
                             (kl_max - kl_min) * progress_frac + kl_min)
        return np.where(batch_idx > kl_warmup_end, kl_max, kl_warmup)
    return kl_warmup


In [ ]:
# Plot the warmup function and the learning rate decay function.
plt.figure(figsize=(16,4))
plt.subplot(121)
x = onp.arange(0, num_batches, print_every)
kl_warmup_fun = get_kl_warmup_fun()
plt.plot(x, [kl_warmup_fun(i) for i in onp.arange(1, num_batches, print_every)]);
plt.title('KL warmup function')
plt.xlabel('Training batch')

plt.subplot(122)
decay_fun = optimizers.exponential_decay(step_size, decay_steps, decay_factor)                                                          
plt.plot(x, [decay_fun(i) for i in range(1, num_batches, print_every)]);
plt.title('learning rate function')
plt.xlabel('Training batch') 


## Randomization
> JAX uses its own setup to handle randomness and seeding the pseudo-random number generators.  You can read about it [here](https://github.com/google/jax/blob/master/README.md#random-numbers-are-different).

In [ ]:
key = random.PRNGKey(onp.random.randint(0, MAX_SEED_INT))

def keygen(key, nkeys):
    """Generate randomness that JAX can use by splitting the JAX keys.
    
    Args:
    key : the random.PRNGKey for JAX
    nkeys : how many keys in key generator
    
    Returns:
    2-tuple (new key for further generators, key generator)
    """
    keys = random.split(key, nkeys+1)
    return keys[0], (k for k in keys[1:])

## Define LFADS functions
These functions are copied directly from the LFADS tutorial [here](https://github.com/google-research/computation-thru-dynamics/blob/master/lfads_tutorial/lfads.py),
with a bit of cleaning up and trimming comments for familiar things.

### Methods for parameter random initialization

In [ ]:
# For linear/affine transforms,
# y = w x, y = w x + b
# with output size o, input size u, and keygen seed.
def linear_params(key, o, u, scale_factor=1.0):
    key, skeys = keygen(key, 1)
    scale_factor = scale_factor / np.sqrt(u)
    return {'w': random.normal(next(skeys), (o, u)) * scale_factor}


def affine_params(key, o, u, scale_factor=1.0):
    key, skeys = keygen(key, 1)
    scale_factor = scale_factor / np.sqrt(u)
    return {'w' : random.normal(next(skeys), (o, u)) * scale_factor,
            'b' : np.zeros((o,))}


def gru_params(key, n, u, ifactor=1.0, hfactor=1.0, hscale=0.0):
    """Generate GRU parameters
    
    Arguments:
    key: random.PRNGKey for random bits
    n: hidden state size
    u: input size
    ifactor: scaling factor for input weights
    hfactor: scaling factor for hidden -> hidden weights
    hscale: scale on h0 initial condition
    
    Returns:
    a dictionary of parameters
    """
    key, skeys = keygen(key, 5)
    ifactor = ifactor / np.sqrt(u)
    hfactor = hfactor / np.sqrt(n)
    
    wRUH = random.normal(next(skeys), (n+n,n)) * hfactor
    wRUX = random.normal(next(skeys), (n+n,u)) * ifactor
    wRUHX = np.concatenate([wRUH, wRUX], axis=1)
    
    wCH = random.normal(next(skeys), (n,n)) * hfactor
    wCX = random.normal(next(skeys), (n,u)) * ifactor
    wCHX = np.concatenate([wCH, wCX], axis=1)
    
    return {'h0' : random.normal(next(skeys), (n,)) * hscale,
            'wRUHX' : wRUHX,
            'wCHX' : wCHX,
            'bRU' : np.zeros((n+n,)),
            'bC' : np.zeros((n,))}


### Transformation implementations
sigmoid $y=\frac{e^x}{1+e^x}=\frac{\tanh(\frac{x}{2}) + 1}{2}$

affine transform $y = w x + b$

(batched) normed linear transform
$y = \hat{w} x$,
where $\hat{w}_{ij} = w_{ij} / |w_{i:}|$

GRU

Dropout (batched)

In [ ]:
def sigmoid(vals):
    return 0.5 * (np.tanh(vals / 2.) + 1)


def affine(params, x):
    """Implements y = w x + b"""
    return np.dot(params['w'], x) + params['b']
batch_affine = vmap(affine, in_axes=(None, 0))


def normed_linear(params, x):
    """Implement y = \hat{w} x, where \hat{w}_ij = w_ij / |w_{i:}|, norm over j
    
    Arguments:
    params: a dictionary of params
    x: np array of input
    
    Returns:
    np array of output
    """
    w = params['w']
    w_row_norms = np.sqrt(np.sum(w**2, axis=1, keepdims=True))
    w = w / w_row_norms
    return np.dot(w, x)
batch_normed_linear = vmap(normed_linear, in_axes=(None, 0))


def gru(params, h, x, bfg=0.5):
    """
    Implement the GRU equations.
    
    Arguments:
    params: dictionary of GRU parameters
    h: np array of  hidden state
    x: np array of input
    bfg: bias on forget gate (useful for learning if > 0.0)
    
    Returns:
    np array of hidden state after GRU update
    """
    hx = np.concatenate([h, x], axis=0)
    ru = sigmoid(np.dot(params['wRUHX'], hx) + params['bRU'])
    r, u = np.split(ru, 2, axis=0)
    rhx = np.concatenate([r * h, x])
    c = np.tanh(np.dot(params['wCHX'], rhx) + params['bC'] + bfg)
    return u * h + (1.0 - u) * c


def dropout(x, key, keep_rate):
    """Implement a dropout layer.
    
    Arguments:
    x: np array to be dropped out
    key: random.PRNGKey for random bits
    keep_rate: dropout rate
    
    Returns:
    np array of dropped out x
    """
    # The shenanigans with np.where are to avoid having to re-jit if
    # keep rate changes.
    do_keep = random.bernoulli(key, keep_rate, x.shape)
    kept_rates = np.where(do_keep, x / keep_rate, 0.0)
    return np.where(keep_rate < 1.0, kept_rates, x)

batch_dropout = vmap(dropout, in_axes=(0, 0, None))


### Runners
These functions run a layer/transformation over a sequence of data.

In [ ]:
def run_dropout(x_t, key, keep_rate):
    """
    Run the dropout layer over additional dimensions, e.g. time.
    
    Arguments:
    x_t: np array to be dropped out
    key: random.PRNGKey for random bits
    keep_rate: dropout rate
    
    Returns:
    np array of dropped out x
    """
    ntime = x_t.shape[0]
    keys = random.split(key, ntime)
    return batch_dropout(x_t, keys, keep_rate)

def run_rnn(params, rnn, x_t, h0=None):
    """
    Run an RNN module forward in time.
    
    Arguments:
    params: dictionary of RNN parameters
    rnn: function for running RNN one step
    x_t: np array data for RNN input with leading dim being time
    h0: initial condition for running rnn, which overwrites param h0
    
    Returns:
    np array of rnn applied to time data with leading dim being time
    """
    h = h0 if h0 is not None else params['h0']
    h_t = []
    for x in x_t:
        h = rnn(params, h, x)
        h_t.append(h)
    return np.array(h_t)


def run_bidirectional_rnn(params, fwd_rnn, bwd_rnn, x_t):
    """
    Run an RNN encoder backwards and forwards over some time series data.
    
    Arguments:
    params: a dictionary of bidrectional RNN encoder parameters
    fwd_rnn: function for running forward rnn encoding
    bwd_rnn: function for running backward rnn encoding
    x_t: np array data for RNN input with leading dim being time
    
    Returns:
    tuple of np array concatenated forward, backward encoding, and
      np array of concatenation of [forward_enc(T), backward_enc(1)]
    """
    fwd_enc_t = run_rnn(params['fwd_rnn'], fwd_rnn, x_t)
    bwd_enc_t = np.flipud(run_rnn(params['bwd_rnn'], bwd_rnn, np.flipud(x_t)))
    full_enc = np.concatenate([fwd_enc_t, bwd_enc_t], axis=1)
    enc_ends = np.concatenate([bwd_enc_t[0], fwd_enc_t[-1]], axis=1)
    return full_enc, enc_ends


### Encoder Part + Decoder Part

In [ ]:
import indl.lfads.distributions as dists

def lfads_encode(params, lfads_hps, key, x_t, keep_rate):
    """Run the LFADS network from input to generator initial condition vars.
    
    Arguments:
    params: a dictionary of LFADS parameters
    lfads_hps: a dictionary of LFADS hyperparameters
    key: random.PRNGKey for random bits
    x_t: np array input for lfads with leading dimension being time
    keep_rate: dropout keep rate
    
    Returns:
    3-tuple of np arrays: generator initial condition mean, log variance
      and also bidirectional encoding of x_t, with leading dim being time
    """
    key, skeys = keygen(key, 3)
    
    # Encode the input
    x_t = run_dropout(x_t, next(skeys), keep_rate)
    con_ins_t, gen_pre_ics = run_bidirectional_rnn(params['ic_enc'], gru, gru, x_t)
    # Push through to posterior mean and variance for initial conditions.
    xenc_t = dropout(con_ins_t, next(skeys), keep_rate)
    gen_pre_ics = dropout(gen_pre_ics, next(skeys), keep_rate)
    ic_gauss_params = affine(params['gen_ic'], gen_pre_ics)
    ic_mean, ic_logvar = np.split(ic_gauss_params, 2, axis=0)
    return ic_mean, ic_logvar, xenc_t


def lfads_decode(params, lfads_hps, key, ic_mean, ic_logvar, xenc_t, keep_rate):
    """Run the LFADS network from latent variables to log rates.
    
    Arguments:
    params: a dictionary of LFADS parameters
    lfads_hps: a dictionary of LFADS hyperparameters
    key: random.PRNGKey for random bits
    ic_mean: np array of generator initial condition mean
    ic_logvar: np array of generator initial condition log variance
    xenc_t: np array bidirectional encoding of input (x_t) with leading dim
      being time
    keep_rate: dropout keep rate
    
    Returns:
    7-tuple of np arrays all with leading dim being time,
      controller hidden state, inferred input mean, inferred input log var,
      generator hidden state, factors and log rates
    """
    
    ntime = lfads_hps['ntimesteps']
    key, skeys = keygen(key, 1+2*ntime)
    
    # Since the factors feed back to the controller,
    #    factors_{t-1} -> controller_t -> sample_t -> generator_t -> factors_t
    # is really one big loop and therefor one RNN.
    c = c0 = params['con']['h0']
    g = g0 = dists.diag_gaussian_sample(next(skeys), ic_mean,
                                      ic_logvar, lfads_hps['var_min'])
    f = f0 = np.zeros((lfads_hps['factors_dim'],))
    c_t = []
    ii_mean_t = []
    ii_logvar_t = []
    ii_t = []
    gen_t = []
    factor_t = []
    for xenc in xenc_t:
        cin = np.concatenate([xenc, f], axis=0)
        c = gru(params['con'], c, cin)
        cout = affine(params['con_out'], c)
        ii_mean, ii_logvar = np.split(cout, 2, axis=0) # inferred input params
        ii = dists.diag_gaussian_sample(next(skeys), ii_mean,
                                        ii_logvar, lfads_hps['var_min'])
        g = gru(params['gen'], g, ii)
        g = dropout(g, next(skeys), keep_rate)
        f = normed_linear(params['factors'], g)
        # Save everything.
        c_t.append(c)
        ii_t.append(ii)
        gen_t.append(g)
        ii_mean_t.append(ii_mean)
        ii_logvar_t.append(ii_logvar)
        factor_t.append(f)
    
    c_t = np.array(c_t)
    ii_t = np.array(ii_t)
    gen_t = np.array(gen_t)
    ii_mean_t = np.array(ii_mean_t)
    ii_logvar_t = np.array(ii_logvar_t)
    factor_t = np.array(factor_t)
    lograte_t = batch_affine(params['logrates'], factor_t)
    
    return c_t, ii_mean_t, ii_logvar_t, ii_t, gen_t, factor_t, lograte_t


def lfads(params, lfads_hps, key, x_t, keep_rate):
    """Run the LFADS network from input to output.
    
    Arguments:
    params: a dictionary of LFADS parameters
    lfads_hps: a dictionary of LFADS hyperparameters
    key: random.PRNGKey for random bits
    x_t: np array of input with leading dim being time
    keep_rate: dropout keep rate
    
    Returns:
    A dictionary of np arrays of all LFADS values of interest.
    """
    
    key, skeys = keygen(key, 2)
    
    ic_mean, ic_logvar, xenc_t = \
        lfads_encode(params, lfads_hps, next(skeys), x_t, keep_rate)
    
    c_t, ii_mean_t, ii_logvar_t, ii_t, gen_t, factor_t, lograte_t = \
        lfads_decode(params, lfads_hps, next(skeys), ic_mean, ic_logvar,
                     xenc_t, keep_rate)
    
    # As this is tutorial code, we're passing everything around.
    return {'xenc_t' : xenc_t, 'ic_mean' : ic_mean, 'ic_logvar' : ic_logvar,
            'ii_t' : ii_t, 'c_t' : c_t, 'ii_mean_t' : ii_mean_t,
            'ii_logvar_t' : ii_logvar_t, 'gen_t' : gen_t, 'factor_t' : factor_t,
            'lograte_t' : lograte_t}


lfads_encode_jit = jit(lfads_encode)
lfads_decode_jit = jit(lfads_decode, static_argnums=(1,))

# Batching accomplished by vectorized mapping.
# We simultaneously map over random keys for forward-pass randomness
# and inputs for batching.
batch_lfads = vmap(lfads, in_axes=(None, None, 0, 0, None))


### Training Functions


In [ ]:
def optimize_lfads_core(key, batch_idx_start, num_batches,
                        update_fun, kl_warmup_fun,
                        opt_state, lfads_hps, lfads_opt_hps, train_data):
    """Make gradient updates to the LFADS model.
    
    Uses lax.fori_loop instead of a Python loop to reduce JAX overhead. This 
    loop will be jit'd and run on device.
    
    Arguments:
    init_params: a dict of parameters to be trained
    batch_idx_start: Where are we in the total number of batches
    num_batches: how many batches to run
    update_fun: the function that changes params based on grad of loss
    kl_warmup_fun: function to compute the kl warmup
    opt_state: the jax optimizer state, containing params and opt state
    lfads_hps: dict of lfads model HPs
    lfads_opt_hps: dict of optimization HPs
    train_data: nexamples x time x ndims np array of data for training
    
    Returns:
    opt_state: the jax optimizer state, containing params and optimizer state"""
    
    key, dkeyg = keygen(key, num_batches) # data
    key, fkeyg = keygen(key, num_batches) # forward pass
    
    # Begin optimziation loop. Explicitly avoiding a python for-loop
    # so that jax will not trace it for the sake of a gradient we will not use.  
    def run_update(batch_idx, opt_state):
        kl_warmup = kl_warmup_fun(batch_idx)
        didxs = random.randint(next(dkeyg), [lfads_hps['batch_size']], 0,
                               train_data.shape[0])
        x_bxt = train_data[didxs].astype(np.float32)
        opt_state = update_fun(batch_idx, opt_state, lfads_hps, lfads_opt_hps,
                               next(fkeyg), x_bxt, kl_warmup)
        return opt_state
    
    lower = batch_idx_start
    upper = batch_idx_start + num_batches
    return lax.fori_loop(lower, upper, run_update, opt_state)


optimize_lfads_core_jit = jit(optimize_lfads_core, static_argnums=(2,3,4,6,7))


def optimize_lfads(key, init_params, lfads_hps, lfads_opt_hps,
                   train_data, eval_data):
    """Optimize the LFADS model and print batch based optimization data.
    
    This loop is at the cpu nonjax-numpy level.
    
    Arguments:
    init_params: a dict of parameters to be trained
    lfads_hps: dict of lfads model HPs
    lfads_opt_hps: dict of optimization HPs
    train_data: nexamples x time x ndims np array of data for training
    
    Returns:
    a dictionary of trained parameters"""
    
    # Begin optimziation loop.
    all_tlosses = []
    all_elosses = []
    
    # Build some functions used in optimization.
    kl_warmup_fun = get_kl_warmup_fun(lfads_opt_hps)
    decay_fun = optimizers.exponential_decay(lfads_opt_hps['step_size'],
                                           lfads_opt_hps['decay_steps'],
                                           lfads_opt_hps['decay_factor'])
    
    opt_init, opt_update, get_params = optimizers.adam(step_size=decay_fun,
                                                     b1=lfads_opt_hps['adam_b1'],
                                                     b2=lfads_opt_hps['adam_b2'],
                                                     eps=lfads_opt_hps['adam_eps'])
    opt_state = opt_init(init_params)
    
    def update_w_gc(i, opt_state, lfads_hps, lfads_opt_hps, key, x_bxt,
                  kl_warmup):
        """Update fun for gradients, includes gradient clipping."""
        params = get_params(opt_state)
        grads = grad(lfads.lfads_training_loss)(params, lfads_hps, key, x_bxt,
                                                kl_warmup,
                                                lfads_opt_hps['keep_rate'])
        clipped_grads = optimizers.clip_grads(grads, lfads_opt_hps['max_grad_norm'])
        return opt_update(i, clipped_grads, opt_state)
    
    # Run the optimization, pausing every so often to collect data and
    # print status.
    batch_size = lfads_hps['batch_size']
    num_batches = lfads_opt_hps['num_batches']
    print_every = lfads_opt_hps['print_every']
    num_opt_loops = int(num_batches / print_every)
    params = get_params(opt_state)
    for oidx in range(num_opt_loops):
        batch_idx_start = oidx * print_every
        start_time = time.time()
        key, tkey, dtkey, dekey = random.split(random.fold_in(key, oidx), 4)
        opt_state = optimize_lfads_core_jit(tkey, batch_idx_start,
                                            print_every, update_w_gc, kl_warmup_fun,
                                            opt_state, lfads_hps, lfads_opt_hps,
                                            train_data)
        batch_time = time.time() - start_time
        
        # Losses
        params = get_params(opt_state)
        batch_pidx = batch_idx_start + print_every
        kl_warmup = kl_warmup_fun(batch_idx_start)
        # Training loss
        didxs = onp.random.randint(0, train_data.shape[0], batch_size)
        x_bxt = train_data[didxs].astype(onp.float32)
        tlosses = lfads.lfads_losses_jit(params, lfads_hps, dtkey, x_bxt,
                                         kl_warmup, 1.0)
        
        # Evaluation loss
        didxs = onp.random.randint(0, eval_data.shape[0], batch_size)
        ex_bxt = eval_data[didxs].astype(onp.float32)
        elosses = lfads.lfads_losses_jit(params, lfads_hps, dekey, ex_bxt,
                                         kl_warmup, 1.0)
        # Saving, printing.
        all_tlosses.append(tlosses)
        all_elosses.append(elosses)
        s = "Batches {}-{} in {:0.2f} sec, Step size: {:0.5f}, Training loss {:0.0f}, Eval loss {:0.0f}"
        print(s.format(batch_idx_start+1, batch_pidx, batch_time,
                       decay_fun(batch_pidx), tlosses['total'], elosses['total']))
        
        tlosses_thru_training = utils.merge_losses_dicts(all_tlosses)
        elosses_thru_training = utils.merge_losses_dicts(all_elosses)
        optimizer_details = {'tlosses' : tlosses_thru_training,
                             'elosses' : elosses_thru_training}
    
    return params, optimizer_details


### Calculate Losses

In [ ]:
def lfads_losses(params, lfads_hps, key, x_bxt, kl_scale, keep_rate):
    """Compute the training loss of the LFADS autoencoder
    
    Arguments:
    params: a dictionary of LFADS parameters
    lfads_hps: a dictionary of LFADS hyperparameters
    key: random.PRNGKey for random bits
    x_bxt: np array of input with leading dims being batch and time
    keep_rate: dropout keep rate
    kl_scale: scale on KL
    
    Returns:
    a dictionary of all losses, including the key 'total' used for optimization
    """
    
    B = lfads_hps['batch_size']
    key, skeys = keygen(key, 2)
    keys = random.split(next(skeys), B)
    lfads = batch_lfads(params, lfads_hps, keys, x_bxt, keep_rate)
    
    # Sum over time and state dims, average over batch.
    # KL - g0
    ic_post_mean_b = lfads['ic_mean']
    ic_post_logvar_b = lfads['ic_logvar']
    kl_loss_g0_b = dists.batch_kl_gauss_gauss(ic_post_mean_b, ic_post_logvar_b,
                                            params['ic_prior'],
                                            lfads_hps['var_min'])
    kl_loss_g0_prescale = np.sum(kl_loss_g0_b) / B  
    kl_loss_g0 = kl_scale * kl_loss_g0_prescale
    
    # KL - Inferred input
    ii_post_mean_bxt = lfads['ii_mean_t']
    ii_post_var_bxt = lfads['ii_logvar_t']
    keys = random.split(next(skeys), B)
    kl_loss_ii_b = dists.batch_kl_gauss_ar1(keys, ii_post_mean_bxt,
                                          ii_post_var_bxt, params['ii_prior'],
                                          lfads_hps['var_min'])
    kl_loss_ii_prescale = np.sum(kl_loss_ii_b) / B
    kl_loss_ii = kl_scale * kl_loss_ii_prescale
    
    # Log-likelihood of data given latents.
    lograte_bxt = lfads['lograte_t']
    log_p_xgz = np.sum(dists.poisson_log_likelihood(x_bxt, lograte_bxt)) / B
    
    # L2
    l2reg = lfads_hps['l2reg']
    l2_loss = l2reg * optimizers.l2_norm(params)**2
    
    loss = -log_p_xgz + kl_loss_g0 + kl_loss_ii + l2_loss
    all_losses = {'total' : loss, 'nlog_p_xgz' : -log_p_xgz,
                'kl_g0' : kl_loss_g0, 'kl_g0_prescale' : kl_loss_g0_prescale,
                'kl_ii' : kl_loss_ii, 'kl_ii_prescale' : kl_loss_ii_prescale,
                'l2' : l2_loss}
    return all_losses


def lfads_training_loss(params, lfads_hps, key, x_bxt, kl_scale, keep_rate):
    """Pull out the total loss for training.
    
    Arguments:
    params: a dictionary of LFADS parameters
    lfads_hps: a dictionary of LFADS hyperparameters
    key: random.PRNGKey for random bits
    x_bxt: np array of input with leading dims being batch and time
    keep_rate: dropout keep rate
    kl_scale: scale on KL
    
    Returns:
    return the total loss for optimization
    """
    losses = lfads_losses(params, lfads_hps, key, x_bxt, kl_scale, keep_rate)
    return losses['total']

from indl.lfads import utils
def posterior_sample_and_average(params, lfads_hps, key, x_txd):
    """Get the denoised lfad inferred values by posterior sample and average.
    
    Arguments:
    params: dictionary of lfads parameters
    lfads_hps: dict of LFADS hyperparameters
    key: JAX random state
    x_txd: 2d np.array time by dim trial to denoise
    
    Returns: 
    LFADS dictionary of inferred values, averaged over randomness.
    """
    batch_size = lfads_hps['batch_size']
    skeys = random.split(key, batch_size)  
    x_bxtxd = np.repeat(np.expand_dims(x_txd, axis=0), batch_size, axis=0)
    lfads_dict = batch_lfads(params, lfads_hps, skeys, x_bxtxd, 1.0)
    return utils.average_lfads_batch(lfads_dict)


### Compilation
>JIT functions are orders of magnitude faster.  The first time you use them,
they will take a couple of minutes to compile, then the second time you use
them, they will be blindingly fast.


In [ ]:
# The static_argnums is telling JAX to ignore the lfads_hps dictionary,
# which means you'll have to pay attention if you change the params.
# How does one force a recompile?
batch_lfads_jit = jit(batch_lfads, static_argnums=(1,))
lfads_losses_jit = jit(lfads_losses, static_argnums=(1,))
lfads_training_loss_jit = jit(lfads_training_loss, static_argnums=(1,))
posterior_sample_and_average_jit = jit(posterior_sample_and_average, static_argnums=(1,))


## Train LFADS

In [ ]:
key = random.PRNGKey(onp.random.randint(0, MAX_SEED_INT))
trained_params, opt_details = \
    optimize_lfads(key, init_params, lfads_hps, lfads_opt_hps, train_data, eval_data)


### Plot training details

### Save the result

## Inspect Model